In [2]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import linear_kernel
from sklearn.tree import DecisionTreeClassifier
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf

from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import re
import nltk 
from nltk.corpus import stopwords
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
from sentence_transformers import SentenceTransformer

## sentence-BERT를 이용한 문장 유사도 측정

In [3]:
### 필요한 라이브러리 설치
!pip install sentence_transformers

In [4]:
### 필요한 라이브러리 임폴트

from sentence_transformers import SentenceTransformer #임폴트 할 때는 _(언더바)

### 영어 문장 유사도 측정

In [5]:
### SentenceTransformer(SentenceBERt) Model Generation

model_name = 'sentence-transformers/all-MiniLM-L12-v2'
model_eng = SentenceTransformer(model_name)


In [6]:
## 영문 텍스트 데이터 생성
eng_sentences = ['What should I do to be a great scientist?', 'How can I be a good scientist?']

# TF벡터라이저, WOrd2Vec임베딩해서 하면 0.20정도 나옴., 기존 왼쪽 방식보다 유사도가 더 떨어짐

In [7]:
### 문장 임베딩 벡터 생성
eng_embeddings = model_eng.encode(eng_sentences)#임베딩할 텍스트(eng_sentences)를 넣으면 임베딩벡터가 나옴 

# 결과 확인하기
print(f'영문 텍스트 데이터의 임베딩 행렬의 모양 : \n{eng_embeddings.shape}')

print('*'*80)

print(f'첫번째 문장의 임베딩 벡터 : \n{eng_embeddings[0,:]}')

print('*'*80)

print(f'두번째 문장의 임베딩 벡터 : \n{eng_embeddings[1,:]}')

영문 텍스트 데이터의 임베딩 행렬의 모양 : 
(2, 384)
********************************************************************************
첫번째 문장의 임베딩 벡터 : 
[ 2.22635996e-02  8.50509554e-02  7.91068673e-02 -6.84862677e-03
 -1.46627296e-02 -6.85858950e-02  2.64204536e-02  6.64388612e-02
 -4.24608216e-02 -3.75801697e-02  1.80893615e-02 -9.48860273e-02
 -4.75542359e-02  7.65433833e-02 -3.93009409e-02  2.58018766e-02
 -7.41133988e-02  5.87386154e-02  3.13343219e-02 -7.86113590e-02
 -6.38961047e-02  4.31189835e-02  6.02601888e-03  1.21794106e-03
 -5.90102263e-02  8.42712224e-02 -2.48904526e-02 -6.01979196e-02
  2.75624357e-02 -5.94473891e-02 -8.06315627e-04 -7.62991458e-02
  1.78551674e-02  2.06335727e-02 -3.27370763e-02  4.25934680e-02
  6.07814975e-02 -2.56904997e-02  5.79137839e-02  5.28666377e-02
  9.59740113e-03 -4.70351651e-02  2.61405464e-02 -9.10787319e-04
  8.14121682e-03 -1.69612914e-02  2.88743693e-02 -1.83992030e-03
  3.87182869e-02  3.14612389e-02 -5.87457865e-02 -7.69814551e-02
 -1.39820725e-01 -6.1

In [8]:
### 코사인 유사도 측정하기

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

from sklearn.metrics.pairwise import cosine_similarity

# 1에 가까운 값이 나올수록 유사하다.(2.384)를 (1,384)로 만들어줄 것 .reshape()

# 1차원 배열 --> 2차원 배열로 변환

eng_embedding1 = eng_embeddings[0,:].reshape((1,384))
eng_embedding2 = eng_embeddings[1,:].reshape((1,384))

# cosine similarity 측정

eng_sim = cosine_similarity(eng_embedding1, eng_embedding2)

print(f'영문 텍스트 데이터의 문장 간 유사도 : \n{eng_sim}')

# TF/IDF는 0.2가 나옴. 유사도가 올라갔음.

영문 텍스트 데이터의 문장 간 유사도 : 
[[0.8954653]]


### 한글 문장 유사도 측정

In [9]:
### 사전 학습된 한국어 SentenceTransformer 모델 생성
model_name='ddobokki/klue-roberta-base-nli-sts'
kor_model = SentenceTransformer(model_name) # pre_trained의 방식이 아님. 일관된 형태가 아니라는게 안타깝다

In [10]:
### 한글 텍스트 데이터 생성
kor_sentences = ["직원이 무단 퇴사를 했는데 손해 배상 청구할 수 있나요?", 
                 "무단 퇴사한 직원에 대한 손해 배상 청구가 가능한가요?"]



In [11]:
### 문장 임베딩 벡터 생성
kor_embeddings = kor_model.encode(kor_sentences)

# 결과 확인하기
print(f'한글 텍스트 데이터 전체 문장에 대한 임베딩 행렬의 모양 : \n{kor_embeddings.shape}')

print('*'*80)

print(f'첫번째 문장에 대한 임베딩 백터 : \n{kor_embeddings[0,:]}')

print('*'*80)

print(f'두번째 문장에 대한 임베딩 벡터 : \n{kor_embeddings[1,:]}')

print('*'*80)

# 대괄호가 하나다. 768가지 측면에서 문장을 비교해보겠다. 1차원 벡터. 대괄호가 2개면 2차원 행렬.

한글 텍스트 데이터 전체 문장에 대한 임베딩 행렬의 모양 : 
(2, 768)
********************************************************************************
첫번째 문장에 대한 임베딩 백터 : 
[-4.99942243e-01 -3.59200746e-01 -7.38405287e-02  4.84219015e-01
 -4.96614575e-01  5.82403421e-01 -4.56289165e-02 -3.81823689e-01
 -3.94545287e-01  2.18352720e-01  2.40236700e-01 -6.53120279e-01
  1.17321879e-01 -4.41113502e-01 -8.18844978e-03 -2.16763198e-01
  1.42988516e-02 -9.46449935e-02  1.55718818e-01 -1.03794485e-01
 -5.95203757e-01 -5.41155815e-01 -6.22601628e-01 -1.00601912e+00
 -2.20479384e-01 -1.32809684e-01 -4.36400957e-02 -4.01158966e-02
  5.56914806e-01  7.56343484e-01  6.72920495e-02  8.65490496e-01
  4.12385538e-02  2.79036313e-01 -1.98162183e-01 -5.00126891e-02
  3.10567290e-01 -3.46614331e-01 -6.20579004e-01  6.81772292e-01
  8.81857425e-02  2.54626155e-01 -4.60292608e-01 -1.45167917e-01
 -6.55803271e-03 -2.85244584e-01  6.48582820e-03  3.51095021e-01
  9.55610797e-02 -1.56446695e-01 -1.90832317e-01 -3.73097539e-01
  2.39245

In [12]:
### 코사인 유사도 측정하기

from sklearn.metrics.pairwise import cosine_similarity

# 1에 가까운 값이 나올수록 유사하다.(2.784)를 (1,784)로 만들어줄 것 .reshape()

# 1차원 배열 --> 2차원 배열로 변환

kor_embedding1 = kor_embeddings[0,:].reshape((1,768))
kor_embedding2 = kor_embeddings[1,:].reshape((1,768)) # 튜플의 형식으로 리스트 취급. 인덱싱 내지는 슬라이싱 가능

# cosine similarity 측정

kor_sim = cosine_similarity(kor_embedding1, kor_embedding2)

print(f'한글 텍스트 데이터의 문장 간 유사도 : \n{kor_sim}')

# TF/IDF는 0.2가 나옴. 유사도가 올라갔음.

한글 텍스트 데이터의 문장 간 유사도 : 
[[0.9606153]]


### 영어 / 한글 문장 유사도 측정


In [13]:
### 다국어 지원 Sentence-BERT 모델 생성
model_name = 'sentence-transformers/multi-qa-distilbert-cos-v1'
multi_model = SentenceTransformer(model_name)

In [14]:
### 텍스트 데이터 생성

sentences = ['What should I do to be a great scientist?', '훌륭한 과학자가 되려면 어떻게 해야 할까요?']

In [15]:
### 문장 임베딩 벡터 생성하기
embeddings = multi_model.encode(sentences)

# 결과 확인하기
print(f'실습용 문장 전체의 임베팅 행렬의 모양 : {embeddings.shape}')

print('*'*80)

print(f'첫번째 문장의 임베딩 벡터 : \n{embeddings[0, :]}')

print('*'*80)

print(f'두번째 문장의 임베딩 벡터 : \n{embeddings[1,:]}')

print('*'*80)

실습용 문장 전체의 임베팅 행렬의 모양 : (2, 768)
********************************************************************************
첫번째 문장의 임베딩 벡터 : 
[ 4.33272719e-02  2.87681911e-02 -2.25619972e-02  2.69756233e-03
  2.85886284e-02 -4.51363809e-02  1.34211397e-02  6.83975872e-03
 -2.85621844e-02 -7.48944432e-02 -9.43056773e-03  2.29911935e-02
  2.06470825e-02  3.65004465e-02  7.41316797e-03  7.02290330e-04
  1.86648071e-02 -9.12581384e-03 -6.94744242e-03  3.07836514e-02
 -2.64755040e-02 -1.96791328e-02 -3.53544392e-03 -8.88182185e-05
  1.54539244e-02 -1.57900061e-02 -2.65403893e-02  4.71877158e-02
  6.02230169e-02  4.74462239e-03 -2.10162438e-02 -3.50181572e-02
 -3.10517307e-02  4.68189754e-02 -8.11047480e-02 -1.65596120e-02
 -3.81736783e-03  4.10237387e-02 -2.55757626e-02 -5.19993482e-04
  2.46242294e-03 -9.64120682e-03 -4.87334691e-02 -2.87258141e-02
 -6.27682135e-02 -4.07510288e-02  5.53259067e-03  5.63171171e-02
  6.49418011e-02  4.03246246e-02  6.69253841e-02 -1.31387860e-02
  3.73931862e-02  4.582

In [16]:
### 코사인 유사도 측정하기

from sklearn.metrics.pairwise import cosine_similarity

# 1에 가까운 값이 나올수록 유사하다. (2,784)를 (1,784)로 만들어줄 것. reshape

# 1차원 배열 --> 2차원 배열로 변환

embedding0 = embeddings[0,:].reshape((1,-1))
embedding1 = embeddings[1,:].reshape((1,-1))

# cosin_similarity() 사용

sim = cosine_similarity(embedding0, embedding1)

print(f'다국어 텍스트의 문장 간 유사도 : \n{sim}')



다국어 텍스트의 문장 간 유사도 : 
[[0.09255006]]


### 다국어 지원 Sentence-BERT 모델 생성

In [17]:
model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
multi_model = SentenceTransformer(model_name)

In [18]:
### 텍스트 데이터 생성

sentences = ['What should I do to be a great scientist?', '훌륭한 과학자가 되려면 어떻게 해야 할까요?']

In [19]:
### 문장 임베딩 벡터 생성하기
embeddings = multi_model.encode(sentences)

# 결과 확인하기
print(f'실습용 문장 전체의 임베팅 행렬의 모양 : {embeddings.shape}')

print('*'*80)

print(f'첫번째 문장의 임베딩 벡터 : \n{embeddings[0, :]}')

print('*'*80)

print(f'두번째 문장의 임베딩 벡터 : \n{embeddings[1,:]}')

print('*'*80)

실습용 문장 전체의 임베팅 행렬의 모양 : (2, 384)
********************************************************************************
첫번째 문장의 임베딩 벡터 : 
[ 5.46487533e-02  4.94343251e-01  4.10707623e-01  1.27668157e-01
 -9.11793038e-02 -5.76901674e-01  1.17372824e-02  1.73615038e-01
 -3.01463813e-01 -1.61016598e-01 -2.14112047e-02 -7.54894257e-01
 -1.01586737e-01  2.17390526e-02 -2.76769966e-01  4.08532098e-03
 -1.66494504e-01 -1.53407529e-01  9.10676364e-03 -2.29964659e-01
 -4.85452652e-01  1.59717619e-01  7.86593035e-02 -1.78895757e-01
 -2.89161295e-01  2.65020996e-01 -1.87272429e-01 -2.74408728e-01
  8.98738801e-02 -1.40582874e-01  1.85039993e-02 -8.13296437e-02
  2.98330039e-01  1.22198565e-02  8.52153078e-02  4.40637112e-01
 -3.57331783e-02 -2.71989971e-01  3.55497599e-01  2.01634869e-01
  3.68284702e-01  8.60847011e-02  6.20844901e-01  1.44933239e-01
 -1.14840753e-01 -4.09185201e-01 -2.46177185e-02 -2.33240709e-01
  7.04460964e-02  2.04685330e-01 -2.15648487e-01 -1.35802656e-01
 -4.60398763e-01 -4.503

In [20]:
### 코사인 유사도 측정하기

from sklearn.metrics.pairwise import cosine_similarity

# 1에 가까운 값이 나올수록 유사하다. (2,784)를 (1,784)로 만들어줄 것. reshape

# 1차원 배열 --> 2차원 배열로 변환

embedding0 = embeddings[0,:].reshape((1,-1))
embedding1 = embeddings[1,:].reshape((1,-1))

# cosin_similarity() 사용

sim = cosine_similarity(embedding0, embedding1)

print(f'다국어 텍스트의 문장 간 유사도 : \n{sim}')

## 한국어와 영어라 뒤섞여있다면 multilingual을 쓰면 정확도가 높아진다.

다국어 텍스트의 문장 간 유사도 : 
[[0.9493035]]


## 문장 유사도를 이용한 추천 시스템
### 실습 개요
- dataset : 2017년 7월 또는 그 이전에 개봉된 영화 45,000편에 대한 각종 정보가
포함되어 있음
- 영화에 대한 줄거리(텍스트 데이터)를 SentenceBERT를 이용하여 행렬로 변환한다.
- 좋아하는 영화의 제목을 입력한다.
- 입력한 영화의 줄거리와 유사한 줄거리를 가지는 영화 제목을 찾아서 추천해준다

### 우리에게 필요한 컬럼 : 제목(title), 줄거리(overview)
'''

### 필요한 라이브러리 / 함수 임폴트



In [21]:
file_path = 'D:\Code\DataSets/movies_metadata.csv '

df = pd.read_csv(file_path)
print(df)

       adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  False                                                NaN         0   
45462  False                                                NaN         0   
45463  False                                                NaN         0   
45464  False                                                NaN         0   
45465  False                                                NaN         0   

                                                  genres  \
0      [{'id': 

C:\Users\human\AppData\Local\Temp\ipykernel_10208\138689798.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [22]:
columns = df.columns

print(columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [23]:
df.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

### 줄거리(overview), 제목(title) 컬럼을 기준으로 누락 데이터 처리

In [29]:
## 팬시 인덱싱 사용 

cleaned_df = df.loc[:, ['title','overview']]

print(cleaned_df) #누락 데이터가 존재하는 행제거, 열은 제거하면 안됨.


print('*'*80)


cleaned_df.dropna(inplace=True)

print(cleaned_df)

print('*'*80)

cleaned_df.reset_index(drop=True, inplace=True)

print(cleaned_df)

                             title  \
0                        Toy Story   
1                          Jumanji   
2                 Grumpier Old Men   
3                Waiting to Exhale   
4      Father of the Bride Part II   
...                            ...   
45461                       Subdue   
45462          Century of Birthing   
45463                     Betrayal   
45464             Satan Triumphant   
45465                     Queerama   

                                                overview  
0      Led by Woody, Andy's toys live happily in his ...  
1      When siblings Judy and Peter discover an encha...  
2      A family wedding reignites the ancient feud be...  
3      Cheated on, mistreated and stepped on, the wom...  
4      Just when George Banks has recovered from his ...  
...                                                  ...  
45461        Rising and falling between a man and woman.  
45462  An artist struggles to finish his work while a...  
45463  When 

In [30]:
### data의 수를 10,000개로 축소하기

df_data = cleaned_df.iloc[:10000, :]

print(df_data)

                                         title  \
0                                    Toy Story   
1                                      Jumanji   
2                             Grumpier Old Men   
3                            Waiting to Exhale   
4                  Father of the Bride Part II   
...                                        ...   
9995                                     Earth   
9996                                        Él   
9997                                La Chienne   
9998                                  Eréndira   
9999  The Private Lives of Elizabeth and Essex   

                                               overview  
0     Led by Woody, Andy's toys live happily in his ...  
1     When siblings Judy and Peter discover an encha...  
2     A family wedding reignites the ancient feud be...  
3     Cheated on, mistreated and stepped on, the wom...  
4     Just when George Banks has recovered from his ...  
...                                                

#### SentenceTransformer Generation

In [31]:
model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
model = SentenceTransformer(model_name)

In [40]:
### 문장 임베딩 벡터 생성

# overview 컬럼 --> 배열 생성

sentences = df_data.loc[:, 'overview'].values

# model.encode() 함수 사용

embeddings = multi_model.encode(sentences)

# 결과 확인하기

#행은 1만개, 줄거리 하나당 얼마의 크기로 임베딩이 됐느냐 임데딩 하나당, 384

print(f'생성된 임베딩 행렬의 모양 : \n{embeddings.shape}')

print('*'*80)

print(f'문장 1개의 임베딩 벡터 : \n{embeddings[0,:].shape}')

print('*'*80)

print(f'문장 2개의 임베딩 벡터 : \n{embeddings[1,:].shape}')

## BERT모델 그 자체에 입력을 만들어서 넣는거였던게 encode_plus()써서 일일이 만들어서 넣었는데
## Sentence모델은 알아서 다해준다. 그 모든 버트를 실행해서, 결과가 다름.
## Bert는 입력 텍스트의 각 단어별 벡터를 다 만드는게 목표
## 그거에 비해서 SentenceBERT는 각 단어별 평균을 만드는게 목표. BERt모델은 다 과정으로 들어가는것임.
## BERT모델의 처리를 다 해버린다. 대신에 결과값은 문장에 대한 하나의 임베딩값을 구해주는거니까 다르다. 
## 용도가 다른것임. 

## BERT는 단어분류, Sentence는 문장분류
## Dense Layer를 붙이면 Bert모델도 같은 역할을 수행할 수 있으나,
## Sentence는 Dense Layer를 쓰지 않음. 분류가 아니라 용도 자체가 달라지는 것.
## 

In [41]:
### 문장 임베딩 벡터 생성하기
embeddings = multi_model.encode(sentences)

# 결과 확인하기
print(f'실습용 문장 전체의 임베팅 행렬의 모양 : {embeddings.shape}')

print('*'*80)

print(f'첫번째 문장의 임베딩 벡터 : \n{embeddings[0, :]}')

print('*'*80)

print(f'두번째 문장의 임베딩 벡터 : \n{embeddings[1,:]}')

print('*'*80)

실습용 문장 전체의 임베팅 행렬의 모양 : (10000, 384)
********************************************************************************
첫번째 문장의 임베딩 벡터 : 
[-0.14804192  0.2500203   0.3174455  -0.08178001  0.17378111  0.18297745
  0.19735156 -0.12237024 -0.17104712  0.20414932  0.05680643  0.19076183
 -0.1229222   0.43755016  0.29919428  0.16207866 -0.0118868  -0.06659345
 -0.29575443  0.33760625 -0.15076633 -0.09829152  0.3198222  -0.04626113
 -0.0580424  -0.06878734 -0.1594206   0.02629805 -0.03609253 -0.04736252
  0.3601988   0.06216668  0.00064946 -0.22916369 -0.05592067  0.06299685
  0.1384913  -0.04153006 -0.17349419 -0.46272215  0.13310432  0.20263828
  0.15396787 -0.1744541  -0.14076686 -0.06373754  0.17024045 -0.23296982
  0.1581071   0.04178443  0.20430708  0.15290703  0.06227873 -0.10009421
  0.29491276  0.44757795  0.0039462   0.13322979 -0.00085037  0.0829566
 -0.09896128 -0.17300843 -0.07240428 -0.00144832  0.4067115  -0.29283765
 -0.3011782  -0.31254214  0.00610332 -0.30328152  0.30176365 -

In [42]:
### 문장 임베딩 해렬 저장
np.save('D:\Code\Deep Learning/embeddings.npy', embeddings) # file_path, data